<a href="https://colab.research.google.com/github/MathRunner7/Product_Sales_Forecasting/blob/main/Product_Sales_Forecasting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# A. Problem Statement and pre-processing and EDA

## 0. Introduction

**Purpose of this project**

Need and Use of Product Sales Forecasting
Effective sales forecasting is fundamental for multiple aspects of retail management and operation, including:
1.	**Inventory Management:** Accurate sales forecasts help ensure that stores maintain optimal inventory levels—enough to meet customer demand without overstocking, which can lead to increased costs or waste, especially in the case of perishable goods.
2.	**Financial Planning:** Forecasting sales allows businesses to estimate future revenue and manage budgets more effectively. This is crucial for allocating resources to areas such as marketing, staffing, and capital investments.
3.	**Marketing and Promotions:** Understanding when sales peaks and troughs are likely to occur enables retailers to plan effective marketing campaigns and promotional offers to boost revenue or manage customer flow.
4.	**Supply Chain Optimization:** Sales forecasts inform production schedules, logistics, and distribution plans, ensuring that products are available where and when they are needed, thereby reducing transportation and storage costs.
5.	**Strategic Decision Making:** Long-term sales forecasting supports broader business strategies, including store expansions, market entry, and other capital expenditures.

Dataset: https://drive.google.com/drive/folders/1fBQ1PlWMho3kHF9qXrD0McZNfpJIcbrn
> Data description

Sr|Column Name|Description
--|--|--
1	|ID| Unique identifier for each record in the dataset.
2	|Store_id| Unique identifier for each store.
3	|Store_Type| Categorization of the store based on its type.
4	|Location_Type| Classification of the store's location (e.g., urban, suburban).
5	|Region_Code| Code representing the geographical region where the store is located.
6	|Date| The specific date on which the data was recorded.
7	|Holiday| Indicator of whether the date was a holiday (1: Yes, 0: No).
8	|Discount| Indicates whether a discount was offered on the given date (Yes/No).
9	|#Order| The number of orders received by the store on the specified day.
10	|Sales| Total sales amount for the store on the given day.


## 1. Dataset Loading

In [71]:
# Import Basic libraries
import pandas as pd
import numpy as np

# Import Visualization libraries
import plotly.graph_objs as go
import plotly.express as px
from plotly.subplots import make_subplots
import matplotlib.pyplot as plt
import seaborn as sns

# Ignore warnings
import warnings
from langcodes import Language
warnings.filterwarnings('ignore')

# Suppress the specific ConvergenceWarning
from statsmodels.tools.sm_exceptions import ConvergenceWarning
warnings.simplefilter("ignore", ConvergenceWarning)

In [72]:
# Read train and test .csv files into Pandas dataframe format
train_url = 'https://drive.google.com/uc?id=1Rfz9d8m_2CGMoQYBNHhgTP9qTAlUcp2-'
test_url = 'https://drive.google.com/uc?id=1UHRbWau_md8x0Wz9cFwGv6LZV0xJxKwj'
train = pd.read_csv(train_url)
test = pd.read_csv(test_url)

In [73]:
train.sample(5)

,ID,Store_id,Store_Type,Location_Type,Region_Code,Date,Holiday,Discount,#Order,Sales
101351,T1101352,275,S4,L1,R1,2018-10-05,0,No,60,32532.00
172214,T1172215,142,S2,L1,R3,2019-04-17,1,Yes,40,28075.41
18408,T1018409,36,S1,L3,R4,2018-02-20,0,Yes,67,41421.00
37661,T1037662,312,S4,L2,R1,2018-04-14,1,No,175,79191.84
7081,T1007082,135,S1,L1,R4,2018-01-20,0,No,58,40014.00


In [74]:
test.sample(5)

,ID,Store_id,Store_Type,Location_Type,Region_Code,Date,Holiday,Discount
14168,T1202509,71,S2,L5,R1,2019-07-09,0,Yes
19907,T1208248,5,S1,L1,R3,2019-07-25,0,Yes
15901,T1204242,104,S1,L1,R2,2019-07-14,0,No
3632,T1191973,67,S2,L5,R1,2019-06-10,0,Yes
9301,T1197642,337,S4,L2,R1,2019-06-26,0,Yes


## 2. Observations on Data

In [75]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 188340 entries, 0 to 188339
Data columns (total 10 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   ID             188340 non-null  object 
 1   Store_id       188340 non-null  int64  
 2   Store_Type     188340 non-null  object 
 3   Location_Type  188340 non-null  object 
 4   Region_Code    188340 non-null  object 
 5   Date           188340 non-null  object 
 6   Holiday        188340 non-null  int64  
 7   Discount       188340 non-null  object 
 8   #Order         188340 non-null  int64  
 9   Sales          188340 non-null  float64
dtypes: float64(1), int64(3), object(6)
memory usage: 14.4+ MB


In [76]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22265 entries, 0 to 22264
Data columns (total 8 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   ID             22265 non-null  object
 1   Store_id       22265 non-null  int64 
 2   Store_Type     22265 non-null  object
 3   Location_Type  22265 non-null  object
 4   Region_Code    22265 non-null  object
 5   Date           22265 non-null  object
 6   Holiday        22265 non-null  int64 
 7   Discount       22265 non-null  object
dtypes: int64(2), object(6)
memory usage: 1.4+ MB


In [77]:
train.Date = pd.to_datetime(train.Date)
test.Date = pd.to_datetime(test.Date)

In [78]:
train.describe().T

,count,mean,min,25%,50%,75%,max,std
Store_id,188340.0,183.0,1.0,92.0,183.0,274.0,365.0,105.366308
Date,188340,2018-09-15 12:00:00.000000256,2018-01-01 00:00:00,2018-05-09 18:00:00,2018-09-15 12:00:00,2019-01-22 06:00:00,2019-05-31 00:00:00,NaN
Holiday,188340.0,0.131783,0.0,0.0,0.0,0.0,1.0,0.338256
#Order,188340.0,68.205692,0.0,48.0,63.0,82.0,371.0,30.467415
Sales,188340.0,42784.327982,0.0,30426.0,39678.0,51909.0,247215.0,18456.708302


In [79]:
test.describe().T

,count,mean,min,25%,50%,75%,max,std
Store_id,22265.0,183.0,1.0,92.0,183.0,274.0,365.0,105.368395
Date,22265,2019-06-30 23:59:59.999999744,2019-06-01 00:00:00,2019-06-16 00:00:00,2019-07-01 00:00:00,2019-07-16 00:00:00,2019-07-31 00:00:00,NaN
Holiday,22265.0,0.032787,0.0,0.0,0.0,0.0,1.0,0.178082


## 3. Handling missing values and Preprocessing

In [103]:
train_null = train.isna().sum().sum()
test_null = test.isna().sum().sum()
print(f'There are {train_null} nulls in train dataset and {test_null} nulls in test dataset.')

There are 0 nulls in train dataset and 0 nulls in test dataset.


In [104]:
# Define dataset type in separate column for train and test
train['Train'] = True
test['Train'] = False

In [105]:
def decorator(func):
  def wrapper(*args, **kwargs):
    print('='*50)
    result = func(*args, **kwargs)
    print('='*50)
    return result
  return wrapper

@decorator
def df_size(df,typ):
  size = df.memory_usage().sum()/(1024**2)
  print(f'Size of {typ} data is: {size:.2f} MB')
  return size

In [121]:
# Combine both the dataset into single dataframe
data = pd.concat([train, test])
raw_size = df_size(data, 'Non-Converted')
data.reset_index(drop=True, inplace=True)
# Change Datatypes to optimize sizes
# Store_id as unsigned integer 16 (Range is 1 to 371)
data.Store_id = data.Store_id.astype('uint16')
# Store_Type, Location_Type, Region_Code as categorical
data.Store_Type = data.Store_Type.astype('category')
data.Location_Type = data.Location_Type.astype('category')
data.Region_Code = data.Region_Code.astype('category')
# Holiday and Discount as Boolean
data.Holiday = data.Holiday.astype('bool')
data.replace({'Discount':{'Yes':True, 'No':False}}, inplace=True)
# Drop unnecessary column Transaction ID
data.pop('ID')
data.set_index('Date', inplace=True)
processed_size = df_size(data, 'Converted')
data.info()

Size of Non-Converted data is: 17.88 MB
Size of Converted data is: 6.43 MB
<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 210605 entries, 2018-01-01 to 2019-07-31
Data columns (total 9 columns):
 #   Column         Non-Null Count   Dtype   
---  ------         --------------   -----   
 0   Store_id       210605 non-null  uint16  
 1   Store_Type     210605 non-null  category
 2   Location_Type  210605 non-null  category
 3   Region_Code    210605 non-null  category
 4   Holiday        210605 non-null  bool    
 5   Discount       210605 non-null  bool    
 6   #Order         188340 non-null  float64 
 7   Sales          188340 non-null  float64 
 8   Train          210605 non-null  bool    
dtypes: bool(3), category(3), float64(2), uint16(1)
memory usage: 6.4 MB


In [122]:
reduction = 100*(raw_size - processed_size)/raw_size
print(f'''Original size of data is: {raw_size:.2f} MB
Size after processing is: {processed_size:.2f} MB
Reduction in size after processing is: {reduction:.2f}%''')

Original size of data is: 17.88 MB
Size after processing is: 6.43 MB
Reduction in size after processing is: 64.04%


In [118]:
data.describe()

,Store_id,Date,#Order,Sales
count,210605.000000,210605,188340.000000,188340.000000
mean,183.000000,2018-10-16 00:00:00,68.205692,42784.327982
min,1.000000,2018-01-01 00:00:00,0.000000,0.000000
25%,92.000000,2018-05-25 00:00:00,48.000000,30426.000000
50%,183.000000,2018-10-16 00:00:00,63.000000,39678.000000
75%,274.000000,2019-03-09 00:00:00,82.000000,51909.000000
max,365.000000,2019-07-31 00:00:00,371.000000,247215.000000
std,105.366279,NaN,30.467415,18456.708302


In [110]:
# Assign index to Exogenous variable dataframe
exog_holiday = data.Holiday
exog_discount = data.Discount
exog = pd.concat([exog_holiday, exog_discount, data.Train], axis=1)
exog

,Holiday,Discount,Train
0,True,True,True
1,True,True,True
2,True,True,True
3,True,True,True
4,True,True,True
...,...,...,...
210600,False,False,False
210601,False,False,False
210602,False,True,False
210603,False,False,False


## 4. Feature Engineering

In [126]:
data.sample(3)

,Store_id,Store_Type,Location_Type,Region_Code,Holiday,Discount,#Order,Sales,Train,Year,Quarter,Month,Day,Week,Weekday,Weekend
Date,,,,,,,,,,,,,,,,
2018-06-30,308,S4,L2,R3,False,False,152.0,104757.0,True,2018,2,6,30,26,Saturday,Weekend
2018-03-06,100,S1,L1,R2,False,False,58.0,35772.0,True,2018,1,3,6,10,Tuesday,Weekday
2018-12-03,64,S3,L1,R2,False,False,65.0,37077.0,True,2018,4,12,3,49,Monday,Weekday


In [125]:
data['Year'] = data.index.year
data['Quarter'] = data.index.quarter
data['Month'] = data.index.month
data['Day'] = data.index.day
data['Week'] = data.index.isocalendar().week
data['Weekday'] = data.index.day_name()
data['Weekend'] = data.Weekday.apply(lambda x: 'Weekend' if x in ['Saturday','Sunday'] else 'Weekday')

In [ ]:
train.shape

In [ ]:
a = train['language'].value_counts().reset_index()
a['name'] = a['language'].apply(lambda code:Language.make(code).maximize().describe()['language'])
a['script'] = a['language'].apply(lambda code:Language.make(code).maximize().describe()['script'])
a['territory'] = a['language'].apply(lambda code:Language.make(code).maximize().describe()['territory'])

## 5. EDA

In [ ]:
#@title **Univariate Analysis**
fig = make_subplots(rows=1, cols=3)
fig.add_trace(go.Histogram(x=train['language']), row=1, col=1)
fig.add_trace(go.Histogram(x=train['access']), row=1, col=2)
fig.add_trace(go.Histogram(x=train['origin']), row=1, col=3)
fig.update_layout(showlegend=False)
fig.show()

In [ ]:
#@title **Bi-variate Analysis**
views = pd.DataFrame(train.sum(skipna=True,numeric_only=True, axis=0))
exogenous_indices = exog[exog['Exog'] == 1].index
plt.figure(figsize=(14, 5))
plt.plot(views.index, views[0])
for date in exogenous_indices:
    plt.axvline(x=date,color='red',alpha=0.7,linewidth=0.4)
plt.show()

In [ ]:
#@title **Hypothesis Testing**
from scipy.stats import chi2_contingency
def chi2test(data, category1, category2, alpha):
  data = train.groupby(by=[category1, category2]).sum().reset_index()
  data['Total']=(data.sum(numeric_only=True, axis=1)/(10**6)).round(2)
  data = data[[category1, category2,'Total']]
  test = chi2_contingency(data.pivot(index=category1,columns=category2,values='Total').fillna(0))
  if test.pvalue < alpha:
    print(f'Reject the Null Hypothesis and {category1} and {category2} are dependent')
  else:
    print(f'Fail to reject the Null Hypothesis and {category1} and {category2} are independent')
  print(f'Test statistics:{test.statistic},\tp-value:{test.pvalue}')
  return None

print('--------------------------------------')
print('Chi2 test between language and access')
chi2test(train, 'language', 'access',0.05)

print('--------------------------------------')
print('Chi2 test between language and origin')
chi2test(train, 'language', 'origin',0.05)

print('--------------------------------------')
print('Chi2 test between access and origin')
chi2test(train, 'access', 'origin',0.05)

## 6. Aggregate and Pivoting

In [ ]:
# Data Preperation for modeling
id_wise = pd.crosstab(index=train.Date, columns=train.Store_id, values =train.Sales, aggfunc='sum')
store_type_wise = pd.crosstab(index=train.Date, columns=train.Store_id, values =train.Sales, aggfunc='sum')
location_wise = pd.crosstab(index=train.Date, columns=train.Store_id, values =train.Sales, aggfunc='sum')
region_wise = pd.crosstab(index=train.Date, columns=train.Store_id, values =train.Sales, aggfunc='sum')

In [ ]:
print(f'List of Languages: {train.language.unique()}')
print(f'List of Access Device: {train.access.unique()}')
print(f'List of Traffic Origin: {train.origin.unique()}')

* We have 128.5k pages after data processing and cleaning, and we cannot 128.5k build a individual forecasting model.
* We have three options to aggregate data
  * Language
  * Access Device
  * Traffic Origin
* Here we will aggregate by summing the view counts.

In [ ]:
# Groupby language code and aggregate by summing
language = train.groupby(by=['language']).sum(numeric_only=True).T
language.index = pd.to_datetime(language.index)
language = language.astype(int)
language.sort_index(axis=1, inplace=True)

# Groupby access device type code and aggregate by summing
access = train.groupby(by=['access']).sum(numeric_only=True).T
access.index = pd.to_datetime(access.index)
access = access.astype(int)
access.sort_index(axis=1, inplace=True)

# Groupby traffic origin code and aggregate by summing
origin = train.groupby(by=['origin']).sum(numeric_only=True).T
origin.index = pd.to_datetime(origin.index)
origin = origin.astype(int)
origin.sort_index(axis=1, inplace=True)

## 7. Time series plots

In [ ]:
#@title Function to plot the data
def plot_timeseries(code):
  fig = go.Figure()
  name = Language.make(code).maximize().describe()['language']
  fig.add_trace(go.Scatter(x=language.index, y=language[code], mode='lines', name=name))
  fig.add_trace(go.Bar(x=language.index, y=exog['Exog'], name='campaign', yaxis='y2',width=50000000, opacity=1))
  fig.update_layout(title=f'{name} Timeseries', showlegend=False, title_x=0.12, title_y=0.75,
      yaxis=dict(title='Views'),
      yaxis2=dict(overlaying='y', showline=False, showgrid=False, showticklabels=False, side='right'))
  return fig

In [ ]:
for code in language.columns:
  plot_timeseries(code).show()

# B. Stationarity, decomposition, detrending, ACF, and PACF

## 8. Stationarity test and decomposition

Most of timeseries model (like **AR, MA, ARIMA**) works on assumption of Stationarity, which makes it easier to predict future values, estimate model parameters, and perform statistical tests. By transforming non-stationary data into a stationary form, analysts can apply a broader range of statistical tools and achieve more reliable results.

To check stationarity of timeseries we will use Augmanted Dickey-Duller test with 5% significance level as threshold.

In [ ]:
# Import ACF/PACF plotting modules
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

# Import Dickey-Fuller test
from statsmodels.tsa.stattools import adfuller

In [ ]:
#@title Print Dickey-Fuller test insights
def adf_test(dataset):
   pvalue = adfuller(dataset)[1]
   if pvalue <= 0.05:
      print('is stationary', end='.')
   else:
      print('is not stationary', end='.')
   print(f'\tp-value is {pvalue}')
for code in language.columns:
  print(f'Time series for "{code}" pages ', end='')
  adf_test(language[code])

### **Decomposition**

In [ ]:
import statsmodels.api as sm
def decompose(series, model_type='additive'):
  decomposition = sm.tsa.seasonal_decompose(series, model=model_type)
  return decomposition.plot()

In [ ]:
language.columns

In [ ]:
# Decomposition for German
model_type='additive'
decompose(language.de, model_type=model_type).show()

In [ ]:
# Decomposition for English
model_type='additive'
decompose(language.en, model_type=model_type).show()

In [ ]:
# Decomposition for Spanish
model_type='additive'
decompose(language.es, model_type=model_type).show()

In [ ]:
# Decomposition for French
model_type='additive'
decompose(language.fr, model_type=model_type).show()

In [ ]:
# Decomposition for Japanese
model_type='additive'
decompose(language.ja, model_type=model_type).show()

In [ ]:
# Decomposition for Russian
model_type='additive'
decompose(language.ru, model_type=model_type).show()

In [ ]:
# Decomposition for Chinese
model_type='additive'
decompose(language.zh, model_type=model_type).show()

## 9. De-trending and de-seasoning

### De-trending

In [ ]:
#@title For detrending we will use differencing by 1 for once and check with Dickey-Fuller test. And do further differencing if there is non-stationarity in data.
language_detrend = language.copy()
difference_order = [1,1,0,0,1,0,1]

for code,d in list(zip(language.columns,difference_order)):
  if d>0:
    language_detrend[code] = language[code].diff(d)

language_detrend.dropna(inplace=True)

for code in language_detrend.columns:
  print(f'Time series for "{code}" pages ', end='')
  adf_test(language_detrend[code])

In [ ]:
#@title Function to compare plot of raw timeseries and detrended timeseries
def detrend(code):
  # Timeseries plot
  plt.figure(figsize=(14, 4))
  language[code].plot()
  plt.title(code)
  plt.figure(figsize=(14, 4))
  language_detrend[code].plot()

In [ ]:
#@title Visualisation of detrended timeseries
for code in language.columns:
  detrend(code)

### De-seasonalising

In [ ]:
#@title For de seasinalising we will take difference by 7 as our seasonality is 7 for each language
language_deseason=language_detrend.diff(7).dropna()

In [ ]:
#@title Plot de-seasonal data
for code in language_deseason.columns:
  plt.figure(figsize=(14, 4))
  language_deseason[code].plot()
  plt.title(code)

## 10. Getting insights into time series characteristics

With the help of ACF and PACF plots model building parameters can be identified
* PACF plot helps to derive `AR(p)` model order `p`
* ACF plot is used to derive `MA(q)` model order `q`

* We will plot ACF plot with de-trended & de-seasoned time series.
* And PACF plot with original time series without de-trending and de-seasoning for each language.


In [ ]:
#@title ACF v/s PACF Plot
for code in language.columns:
  fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))
  # Plot ACF on the left side
  plot_acf(language_deseason[code], ax=ax1)
  ax1.set_title(f'ACF for {code}')

  # Plot PACF on the right side
  plot_pacf(language_deseason[code], ax=ax2)
  ax2.set_title(f'PACF for {code}')

  # Adjust layout
  plt.tight_layout()
  plt.show()

Language|ACF|PACF
--|--|--
German  |Cuts off after lag2            |Cuts off after lag 3
	      |MA(1)/MA(2) component present  |AR(1)/AR(2)/AR(3) component present
English |Cut off at 1 only              |Cuts off at 1 only
	      |Max MA(1) component not present|Max AR(1) component present
Spanish	|Decaying upto lag 5			      |Cuts off after lag 1
	      |MA(1)...MA(5) component present|AR(1) component presentv
French	|Decaying upto lag 4			      |Cuts off after lag 1
	      |MA(1)...MA(4) component present|AR(1) component present
Japanese|Cuts off after lag3            |Cuts off after lag 2
	      |MA(1) component present				|AR(1)/AR(2) component present
Russian	|Decaying upto lag 5			      |Cuts off after lag 2
	      |MA(1)...MA(5) component present|AR(1)/AR(2) component present
Chinese |Cuts off after lag3            |Cuts off after lag 3
	      |MA(1)/MA(2)/MA(3) component present |AR(1)/AR(2)/AR(3) component present


# C. Model building and Evaluation

## 11. Data splitting

To build a model we will split data into 80:20 ratio. Out of 550 days, first 440 days will be considered as train data and remaining 110 days will be used as test data.

In [ ]:
train_threshold = int(language.shape[0]*0.8)
train = language[:train_threshold]
test = language[train_threshold:]
train = train.asfreq('D')
test = test.asfreq('D')

exog.set_index(pd.to_datetime(exog.index), inplace=True)
exog_train = exog[:train_threshold]
exog_test = exog[train_threshold:]
exog_train = exog_train.asfreq('D')
exog_test = exog_test.asfreq('D')

prediction_count = test.shape[0]

## 12. ARIMA model

### Basic model building preperation

In [ ]:
#@title Import performance metrics MSE, MAE, MAPE
from sklearn.metrics import (mean_squared_error as mse, mean_absolute_error as mae, mean_absolute_percentage_error as mape)

# Import ARIMA
from statsmodels.tsa.statespace.sarimax import SARIMAX

# Import itertools for grid generation
import itertools

In [ ]:
#@title Creating a function to print values of all metrics.
def performance(actual, predicted):
    print('MAE :', round(mae(actual, predicted), 3))
    print('RMSE :', round(mse(actual, predicted)**0.5, 3))
    print('MAPE:', round(mape(actual, predicted), 3))
    return round(mape(actual, predicted), 3)
def mape_score(actual, predicted):
    return round(mape(actual, predicted), 3)

In [ ]:
#@title Define the parameter grid
p,d,q = range(0,8), [0,1], range(0, 5)

# Generate all different combinations of p, d, q for each language model
parameters = list(itertools.product(p, d, q))

german_parameters = list(itertools.product([1,2], [1], [1,2,3]))
english_parameters = list(itertools.product([0,1], [1], [0,1]))
spanish_parameters = list(itertools.product([1,2,3,4,5], [0], [0,1]))
french_parameters = list(itertools.product([1,2,3,4], [0], [1]))
japanese_parameters = list(itertools.product([1,2,3,4,5,6], [1], [1,2]))
russian_parameters = list(itertools.product([1,2,3,4,5], [0], [1,2]))
chinese_parameters = list(itertools.product([1,2,3], [1], [1,2,3]))
model_parameters=[german_parameters,english_parameters,spanish_parameters,spanish_parameters,french_parameters,japanese_parameters,russian_parameters,chinese_parameters]

In [ ]:
#@title Function for building best ARIMA model
def bestARIMA(model_language,non_seasonal_order):
  # Initialize a DataFrame to store the results
  results = []
  # Fit the ARIMA model for each combination of parameters
  for param in non_seasonal_order:
    try:
      arima_model = SARIMAX(train[model_language],
                      order=param,
                      enforce_stationarity=False,
                      enforce_invertibility=False).fit(disp=False)
      test[f'{model_language}_pred']=arima_model.forecast(steps=prediction_count)
      # Store the results in the DataFrame
      results.append((param,mape_score(test[model_language],test[f'{model_language}_pred']), arima_model.aic))
    except Exception as e:
      continue
  results_df = pd.DataFrame(results, columns=['params', 'MAPE', 'AIC'])
  results_df.sort_values(by=['MAPE','AIC'], inplace=True)

  # Find the best parameters
  best_params = results_df.iloc[0]
  print(f'''Best ARIMA Model for {model_language} language
  (p,d,q):{best_params.params}
  MAPE:\t{best_params.MAPE}
  AIC:\t{best_params.AIC}''')

  # Fit the best model
  return SARIMAX(train[model_language], order=best_params.params, enforce_stationarity=False, enforce_invertibility=False).fit(disp=False)

In [ ]:
#@title Build best fit ARIMA model for each language
# Create an empty dictionary for forecasting models
best_ARIMA_model = {}

# Build a best ARIMA forecasting model for each language from multiple combinations
for model_language,non_seasonal_order in zip(language.columns,model_parameters):
  best_ARIMA_model[model_language] = bestARIMA(model_language,non_seasonal_order)
  #best_ARIMA_model[model_language] = bestARIMA(model_language,parameters)

In [ ]:
best_ARIMA_model

### Model Summary and Visual Representation

#### All Model's Summary

In [ ]:
#@title Model summary
for model_language in language.columns:
  print('\n','*'*150,'\n','*'*150)
  print('Summaary of ', model_language)
  print(best_ARIMA_model[model_language].summary())

#### All Model's Graphs

In [ ]:
#@title Graphical representation of model
for model_language in language.columns[:2]:
  print('\n','*'*150,'\n','*'*150)
  print('Graphical representation of ', model_language)
  test[f'{model_language}_pred']=best_ARIMA_model[model_language].forecast(steps=prediction_count)
  plt.figure(figsize=(15, 5))
  plt.plot(test[[model_language,f'{model_language}_pred']])
  plt.plot(train[[model_language]])
  best_ARIMA_model[model_language].plot_diagnostics(figsize=(15, 10)).show()
  plt.show()

## 13. SARIMAX model

To  have better accuracy in forecasting we will use exogenous variable

### Basic model building preperation

In [ ]:
#@title Define the parameter grid
p,d,q = range(0,6), [0,1], range(0, 3)
parameters = list(itertools.product(p, d, q))

# seasonal_parameters P, D, Q, S
seasonal_parameters = [(1,0,1,7),(2,0,1,7),(3,0,1,7)]

In [ ]:
#@title Function for building best SARIMAX model
def bestSARIMAX(model_language,parameters,seasonal_parameters):
  # Initialize a DataFrame to store the results
  results = []
  # Fit the SARIMAX model for each combination of parameters
  for param in parameters:
      for seasonal_param in seasonal_parameters:
          try:
              SARIMAX_model = SARIMAX(train[model_language],exog=exog_train,
                              order=param,
                              seasonal_order=seasonal_param,
                              enforce_stationarity=False,
                              enforce_invertibility=False).fit(disp=False)

              # Store the results in the DataFrame
              test[f'{model_language}_pred']=SARIMAX_model.forecast(steps=prediction_count, exog=exog_test)
              results.append((param, seasonal_param, mape_score(test[model_language],test[f'{model_language}_pred']), SARIMAX_model.aic))

          except Exception as e:
              continue

  results_df = pd.DataFrame(results, columns=['params', 'seasonal_params', 'MAPE', 'AIC'])
  results_df.sort_values(by=['MAPE','AIC'], inplace=True)
  # Find the best parameters
  best_params = results_df.iloc[0]

  print(f'''Best SARIMAX Model for {model_language} language
  (p,d,q):{best_params.params}, (P,D,Q,S):{best_params.seasonal_params},
  MAPE:\t{best_params.MAPE}
  AIC:\t{best_params.AIC}''')

  # Fit the best model
  return SARIMAX(train[model_language],exog=exog_train, order=best_params.params, seasonal_order=best_params.seasonal_params, enforce_stationarity=False, enforce_invertibility=False).fit(disp=False)

In [ ]:
#@title Finding best model

best_SARIMAX_model = {}
for model_language,non_seasonal_order in zip(language.columns,model_parameters):
  #best_SARIMAX_model[model_language] = bestSARIMAX(model_language,non_seasonal_order,seasonal_parameters)
  best_SARIMAX_model[model_language] = bestSARIMAX(model_language,parameters,seasonal_parameters)

In [ ]:
#@title Final Model Building
best_fit_model_parameters = {
    'de':[[(5,1,2)],[(1,0,1,7)]],
    'en':[[(4,1,2)],[(1,0,1,7)]],
    'es':[[(3,1,2)],[(2,0,1,7)]],
    'fr':[[(3,1,2)],[(1,0,1,7)]],
    'ja':[[(5,1,2)],[(3,0,1,7)]],
    'ru':[[(1,1,0)],[(2,0,1,7)]],
    'zh':[[(1,1,1)],[(3,0,1,7)]]}

for model_language in language.columns:
#  print(model_language,best_fit_model_parameters[model_language][0],best_fit_model_parameters[model_language][1])
  best_SARIMAX_model[model_language] = bestSARIMAX(model_language,best_fit_model_parameters[model_language][0],best_fit_model_parameters[model_language][1])

For all possible combinations, MAPE score of SARIMAX model is very high for `es` language code, this means `EXOG` variable is not significant. So we will continue without `EXOG` variable for `es` model.

In [ ]:
#@title "es" model building without exogenous variable
def bestSARIMAXnonExog(model_language,parameters,seasonal_parameters):
  # Initialize a DataFrame to store the results
  results = []
  # Fit the SARIMAX model for each combination of parameters
  for param in parameters:
      for seasonal_param in seasonal_parameters:
          try:
              SARIMAX_model = SARIMAX(train[model_language],
                              order=param,
                              seasonal_order=seasonal_param,
                              enforce_stationarity=False,
                              enforce_invertibility=False).fit(disp=False)

              # Store the results in the DataFrame
              test[f'{model_language}_pred']=SARIMAX_model.forecast(steps=prediction_count)
              results.append((param, seasonal_param, mape_score(test[model_language],test[f'{model_language}_pred']), SARIMAX_model.aic))

          except Exception as e:
              continue

  results_df = pd.DataFrame(results, columns=['params', 'seasonal_params', 'MAPE', 'AIC'])
  results_df.sort_values(by=['MAPE','AIC'], inplace=True)
  # Find the best parameters
  best_params = results_df.iloc[0]

  print(f'''Best SARIMAX Model for {model_language} language
  (p,d,q):{best_params.params}, (P,D,Q,S):{best_params.seasonal_params},
  MAPE:\t{best_params.MAPE}
  AIC:\t{best_params.AIC}''')

  # Fit the best model
  return SARIMAX(train[model_language], order=best_params.params, seasonal_order=best_params.seasonal_params, enforce_stationarity=False, enforce_invertibility=False).fit(disp=False)

best_SARIMAX_model['es'] = bestSARIMAXnonExog('es',parameters,seasonal_parameters)

In [ ]:
best_SARIMAX_model

### Model Summary and Visual Representation

#### German Model

In [ ]:
#@title Graphical representation of model
model_language='de'
print(best_SARIMAX_model[model_language].summary())
print('===================================================================================')

test[f'{model_language}_pred']=best_SARIMAX_model[model_language].forecast(steps=prediction_count, exog=exog_test)
plt.figure(figsize=(15, 5))
plt.plot(test[[model_language,f'{model_language}_pred']], label=['test','pred'])
plt.plot(train[[model_language]], label='train')
plt.legend()
best_SARIMAX_model[model_language].plot_diagnostics(figsize=(15, 10)).show()
plt.show()

#### English Model

In [ ]:
#@title Graphical representation of model
model_language='en'
print(best_SARIMAX_model[model_language].summary())
print('===================================================================================')

test[f'{model_language}_pred']=best_SARIMAX_model[model_language].forecast(steps=prediction_count, exog=exog_test)
plt.figure(figsize=(15, 5))
plt.plot(test[[model_language,f'{model_language}_pred']], label=['test','pred'])
plt.plot(train[[model_language]], label='train')
plt.legend()
best_SARIMAX_model[model_language].plot_diagnostics(figsize=(15, 10)).show()
plt.show()

#### Spanish Model

In [ ]:
#@title Graphical representation of model
model_language='es'
print(best_SARIMAX_model[model_language].summary())
print('===================================================================================')

test[f'{model_language}_pred']=best_SARIMAX_model[model_language].forecast(steps=prediction_count)
plt.figure(figsize=(15, 5))
plt.plot(test[[model_language,f'{model_language}_pred']], label=['test','pred'])
plt.plot(train[[model_language]], label='train')
plt.legend()
plt.show()
best_SARIMAX_model[model_language].plot_diagnostics(figsize=(15, 10)).show()

#### French Model

In [ ]:
#@title Graphical representation of model
model_language='fr'
print(best_SARIMAX_model[model_language].summary())
print('===================================================================================')

test[f'{model_language}_pred']=best_SARIMAX_model[model_language].forecast(steps=prediction_count, exog=exog_test)
plt.figure(figsize=(15, 5))
plt.plot(test[[model_language,f'{model_language}_pred']], label=['test','pred'])
plt.plot(train[[model_language]], label='train')
plt.legend()
plt.show()
best_SARIMAX_model[model_language].plot_diagnostics(figsize=(15, 10)).show()

#### Japanese Model

In [ ]:
#@title Graphical representation of model
model_language='ja'
print(best_SARIMAX_model[model_language].summary())
print('===================================================================================')

test[f'{model_language}_pred']=best_SARIMAX_model[model_language].forecast(steps=prediction_count, exog=exog_test)
plt.figure(figsize=(15, 5))
plt.plot(test[[model_language,f'{model_language}_pred']], label=['test','pred'])
plt.plot(train[[model_language]], label='train')
plt.legend()
plt.show()
best_SARIMAX_model[model_language].plot_diagnostics(figsize=(15, 10)).show()

#### Russian Model

In [ ]:
#@title Graphical representation of model
model_language='ru'
print(best_SARIMAX_model[model_language].summary())
print('===================================================================================')

test[f'{model_language}_pred']=best_SARIMAX_model[model_language].forecast(steps=prediction_count, exog=exog_test)
plt.figure(figsize=(15, 5))
plt.plot(test[[model_language,f'{model_language}_pred']], label=['test','pred'])
plt.plot(train[[model_language]], label='train')
plt.legend()
plt.show()
best_SARIMAX_model[model_language].plot_diagnostics(figsize=(15, 10)).show()

#### Chinese Model

In [ ]:
#@title Graphical representation of model
model_language='zh'
print(best_SARIMAX_model[model_language].summary())
print('===================================================================================')

test[f'{model_language}_pred']=best_SARIMAX_model[model_language].forecast(steps=prediction_count, exog=exog_test)
plt.figure(figsize=(15, 5))
plt.plot(test[[model_language,f'{model_language}_pred']], label=['test','pred'])
plt.plot(train[[model_language]], label='train')
plt.legend()
plt.show()
best_SARIMAX_model[model_language].plot_diagnostics(figsize=(15, 10)).show()

## 14. Facebook Prophet

In SARIMAX Model we cannot have multiple seasonality or multiple exogenous variable, we can only select one value. To overcome this disability, we will use an open source library called `prophet` which is developed by Facebook

**Some features of Facebook's Prophet:**
* Provides intuitive parameters which can be easily tuned
* It is robust to missing data and shifts in the trend, and typically handles outliers well.
* It can account for multiple seasonalities. This is possible because under the hood, the math of seasonalities is based on Fourier transforms, which help incorporate this.
* The Prophet uses a decomposable time series model with three main model components
* They are combined in the following equation:
        y(t)= g(t) + s(t) + h(t) + εt

  **g(t):** piece wise linear or logistic growth curve for modeling non-periodic changes in time series (**trend**)

  **s(t):** periodic changes (e.g. weekly/yearly **seasonality**)

  **h(t):** effects of **holidays** (user provided) with irregular schedules

  **εt:** **error term** accounts for any unusual changes not accommodated by the model.

### Import Prophet Library and Prepare Data

In [ ]:
from prophet import Prophet

In [ ]:
df = language.copy()
df['campaign'] = exog
df.reset_index(inplace=True)
df.rename(columns={'index':'ds'}, inplace=True)

### German Model

In [ ]:
model_language = 'de'
df['y'] = df[model_language]
df1 = df[['ds','y','campaign']]
model=Prophet(interval_width=0.95, yearly_seasonality=True, weekly_seasonality=True,
               changepoint_prior_scale=4)
model.add_regressor('campaign')
model.fit(df[:train_threshold])
forecast = model.predict(df)
fig = model.plot(forecast)

In [ ]:
print(f'Performance Score for {model_language} is:')
performance(df1['y'],forecast['yhat'])
plt.figure(figsize=(15, 5))
plt.plot(forecast['ds'], forecast['yhat'],'-*', label = 'Predictions')
plt.plot(df['ds'], df['y'], label = 'Actual')
plt.xlim(pd.to_datetime('2017-01-01'), pd.to_datetime('2017-04-22'))
for x in df.query('campaign==1')['ds']:
    plt.axvline(x=x, color='red', alpha = 0.5);
plt.xlim(pd.to_datetime('2016-08-01'), pd.to_datetime('2016-12-31'))
plt.legend()

### English Model

In [ ]:
model_language = 'en'
df['y'] = df[model_language]
df1 = df[['ds','y','campaign']]
model=Prophet(interval_width=0.95, yearly_seasonality=True, weekly_seasonality=True,
               changepoint_prior_scale=1)
model.add_regressor('campaign')
model.fit(df[:train_threshold])
forecast = model.predict(df)
fig = model.plot(forecast)

In [ ]:
print(f'Performance Score for {model_language} is:')
performance(df1['y'],forecast['yhat'])
plt.figure(figsize=(15, 5))
plt.plot(forecast['ds'], forecast['yhat'],'-*', label = 'Predictions')
plt.plot(df['ds'], df['y'], label = 'Actual')
for x in df.query('campaign==1')['ds']:
    plt.axvline(x=x, color='red', alpha = 0.5);
plt.xlim(pd.to_datetime('2016-08-01'), pd.to_datetime('2016-12-31'))
plt.legend()

### Spanish Model

In [ ]:
model_language = 'es'
df['y'] = df[model_language]
df1 = df[['ds','y','campaign']]
model=Prophet(interval_width=0.95, yearly_seasonality=True, weekly_seasonality=True,
               changepoint_prior_scale=1)
model.add_regressor('campaign')
model.fit(df[:train_threshold])
forecast = model.predict(df)
fig = model.plot(forecast)

In [ ]:
print(f'Performance Score for {model_language} is:')
performance(df1['y'],forecast['yhat'])
plt.figure(figsize=(15, 5))
plt.plot(forecast['ds'], forecast['yhat'],'-*', label = 'Predictions')
plt.plot(df['ds'], df['y'], label = 'Actual')
for x in df.query('campaign==1')['ds']:
    plt.axvline(x=x, color='red', alpha = 0.5);
plt.xlim(pd.to_datetime('2016-08-01'), pd.to_datetime('2016-12-31'))
plt.legend()

### French Model

In [ ]:
model_language = 'fr'
df['y'] = df[model_language]
df1 = df[['ds','y','campaign']]
model=Prophet(interval_width=0.95, yearly_seasonality=True, weekly_seasonality=True,
               changepoint_prior_scale=1)
model.add_regressor('campaign')
model.fit(df[:train_threshold])
forecast = model.predict(df)
fig = model.plot(forecast)

In [ ]:
print(f'Performance Score for {model_language} is:')
performance(df1['y'],forecast['yhat'])
plt.figure(figsize=(15, 5))
plt.plot(forecast['ds'], forecast['yhat'],'-*', label = 'Predictions')
plt.plot(df['ds'], df['y'], label = 'Actual')
plt.xlim(pd.to_datetime('2017-01-01'), pd.to_datetime('2017-04-22'))
for x in df.query('campaign==1')['ds']:
    plt.axvline(x=x, color='red', alpha = 0.5);
plt.xlim(pd.to_datetime('2016-08-01'), pd.to_datetime('2016-12-31'))
plt.legend()

### Japanese Model

In [ ]:
model_language = 'ja'
df['y'] = df[model_language]
df1 = df[['ds','y','campaign']]
model=Prophet(interval_width=0.95, yearly_seasonality=True, weekly_seasonality=True,
               changepoint_prior_scale=1)
model.add_regressor('campaign')
model.fit(df[:train_threshold])
forecast = model.predict(df)
fig = model.plot(forecast)

In [ ]:
print(f'Performance Score for {model_language} is:')
performance(df1['y'],forecast['yhat'])
plt.figure(figsize=(15, 5))
plt.plot(forecast['ds'], forecast['yhat'],'-*', label = 'Predictions')
plt.plot(df['ds'], df['y'], label = 'Actual')
plt.xlim(pd.to_datetime('2017-01-01'), pd.to_datetime('2017-04-22'))
for x in df.query('campaign==1')['ds']:
    plt.axvline(x=x, color='red', alpha = 0.5);
plt.xlim(pd.to_datetime('2016-08-01'), pd.to_datetime('2016-12-31'))
plt.legend()

### Russian Model

In [ ]:
model_language = 'ru'
df['y'] = df[model_language]
df1 = df[['ds','y','campaign']]
model=Prophet(interval_width=0.95, yearly_seasonality=True, weekly_seasonality=True,
               changepoint_prior_scale=1)
model.add_regressor('campaign')
model.fit(df[:train_threshold])
forecast = model.predict(df)
fig = model.plot(forecast)

In [ ]:
print(f'Performance Score for {model_language} is:')
performance(df1['y'],forecast['yhat'])
plt.figure(figsize=(15, 5))
plt.plot(forecast['ds'], forecast['yhat'],'-*', label = 'Predictions')
plt.plot(df['ds'], df['y'], label = 'Actual')
plt.xlim(pd.to_datetime('2017-01-01'), pd.to_datetime('2017-04-22'))
for x in df.query('campaign==1')['ds']:
    plt.axvline(x=x, color='red', alpha = 0.5);
plt.xlim(pd.to_datetime('2016-08-01'), pd.to_datetime('2016-12-31'))
plt.legend()

### Chinese Model

In [ ]:
model_language = 'zh'
df['y'] = df[model_language]
df1 = df[['ds','y','campaign']]
model=Prophet(interval_width=0.95, yearly_seasonality=True, weekly_seasonality=True,
               changepoint_prior_scale=1)
model.add_regressor('campaign')
model.fit(df[:train_threshold])
forecast = model.predict(df)
fig = model.plot(forecast)

In [ ]:
print(f'Performance Score for {model_language} is:')
performance(df1['y'],forecast['yhat'])
plt.figure(figsize=(15, 5))
plt.plot(forecast['ds'], forecast['yhat'],'-*', label = 'Predictions')
plt.plot(df['ds'], df['y'], label = 'Actual')
plt.xlim(pd.to_datetime('2017-01-01'), pd.to_datetime('2017-04-22'))
for x in df.query('campaign==1')['ds']:
    plt.axvline(x=x, color='red', alpha = 0.5);
plt.xlim(pd.to_datetime('2016-08-01'), pd.to_datetime('2016-12-31'))
plt.legend()

# Case Study Insights:

**Problem Description**
* We have total 145K wikipedia pages with pageviews for **550** days. Starting from **2015-07-01** to **2016-12-31**

* There are 15455 pages in this dataset for which more than 30% of the data is missing. Those pages need to be dropped from the dataset. After dropping 129.6k pages are there to process further.

* For remaining Null values following strategy is used for imputation
  * Back-filling of initial null values
  * Linear interpolation for null values in between actual values
  * Forward-filling of tailing null values

* Apart from top 7 languages other language pages are less than 100 in numbers and among 129.6k pages, consolidated 1109 (0.86%) pages which can be considered as outliers. These pages can be dropped. After dropping them 128.5k pages we have for model building.

* List of Languages: [zh, fr, en, ru, de, ja, es]
* List of Access Device: [all-access, desktop, mobile-web]
* List of Traffic Origin: [spider, all-agents]

**Insights from EDA**
1. Among all the languages **_English_** have highest views and other languages have almost similar views
2. Half of the views coming from **_all-access_** device and remaining half views are splitted between **_desktop_** and **_mobile_**
3. 75% of web traffic is generated from **_all-agents_** origin followed by 25% from **_Spider_**
4. Whenever there is campaign (1 in exogenous variable) overall web traffic is very high
5. From hypothesis test it is evident that **language, access and origin** are not independent on each other

**Aggregating and Pivoting**
* We have 128.5k pages after data processing and cleaning, and we cannot 128.5k build a individual forecasting model.
* We have three options to aggregate data and aggregation method should be summing
  * Language
  * Access Device
  * Traffic Origin

* For this case study we will build a forecasting models based on languages

**Insights from timeseries plot**

All timeseries have seasonality of 7 days
1. **German:** There are multiple trends (Up and Down) in this time series
2. **English:** This is an increasing trend time series and very high count on campaign days (exogenous variable)
3. **Spanish:** Additional seasonality is 6 months but no effect of campaign
4. **French:** Initial 6 months is there is minor increasing trend but overall timeseries is stationary with very high values in month of April 2016
5. **Japanese:** This timeseris having high variance with change point in January 2016
6. **Russian:** Apart from effect of campaign between Mid-July to Mid-August 2016, this timeseries is stationary and variance is very low
7. **Chinese:** This timeseries also have no effect of campaign

**Dickey Fuller test** for stationarity with 95% confidence interval

Result of adfuller test is as below which matches with observation of timeseries plot too.
* Detrending is done by 1 difference gives us stationary timeseries for non-stationary timeseries, whereas differencing for stationary timeseries is not requiired.
* De-seasoning to be done by 7 as we have seasonality of 7 in our data.

-|Before Detrending||After Detrending|-
--|--|--|--|--
**Timeseries**|**Stationarity**|**p-value**|**Stationarity**|**p-value**
de|not stationary|0.130302922801|stationary|3.63472666E-10
en|not stationary|0.132635979027|stationary|4.87926062E-13
es|stationary|0.039148315767|stationary|0.037995918905
fr|stationary|0.032074345937|stationary|0.027705575705
ja|not stationary|0.061022041526|stationary|0.0
ru|stationary|0.001661966259|stationary|0.001733881900
zh|not stationary|0.188011855804|stationary|3.51291514E-11

**Insights from ACF/PACF Plots**

Language|ACF|PACF
--|--|--
German  |Cuts off after lag2            |Cuts off after lag 3
	      |MA(1)/MA(2) component present  |AR(1)/AR(2)/AR(3) component present
English |Cut off at 1 only              |Cuts off at 1 only
	      |Max MA(1) component not present|Max AR(1) component present
Spanish	|Decaying upto lag 5			      |Cuts off after lag 1
	      |MA(1)...MA(5) component present|AR(1) component presentv
French	|Decaying upto lag 4			      |Cuts off after lag 1
	      |MA(1)...MA(4) component present|AR(1) component present
Japanese|Cuts off after lag3            |Cuts off after lag 2
	      |MA(1) component present				|AR(1)/AR(2) component present
Russian	|Decaying upto lag 5			      |Cuts off after lag 2
	      |MA(1)...MA(5) component present|AR(1)/AR(2) component present
Chinese |Cuts off after lag3            |Cuts off after lag 3
	      |MA(1)/MA(2)/MA(3) component present |AR(1)/AR(2)/AR(3) component present


**Prelimnary estimate of Non-Seasonal Order from ACF/PACF charts**

Language|AR(p)|integration(d)|MA(q)
--|:--:|:--:|:--:
**German:**|1,2|1|1,2
**English:**|0,1|1|0,1
**Spanish:**|1,2,3,4,5|0|1
**French:**|1,2,3,4|0|1
**Japanese:**|1,2,3,4,5,6|1|1,2
**Russian:**|1,2,3,4,5|0|1,2
**Chinese:**|1,2,3|1|1,2,3

**ARIMA Model** uses Autoregression, Moving Average and Integration of differenciated timeseries. We are getting stationary timeseries by difference of 1 in German, English, Japanese and Chinese. But for simplicity of modeling, we have differenciated all the timeseries by 1 so value of **`d=1`** shall be in all the models. Value of **`p`** and **`q`** already have been derived from ACF and PACF plots and with the help of grid search concept best fit ARIMA model parameters are as follows.

**Language**|**(p,d,q)**|**MAPE**
--|:--:|:--:
**German:**|(2,1,3)|0.099|(1,1,2)|(1,0,1,7)|0.058
**English:**|(0,1,0)|0.069|(4,1,2)|(1,0,1,7)|0.063
**Spanish:**|(4,0,0)|0.138|(3,1,2)|(2,0,1,7)|0.315
**French:**|(5,0,1)|0.073|(3,1,2)|(1,0,1,7)|0.071
**Japanese:**|(2,0,1)|0.091|(6,1,1)|(3,0,1,7)|0.062
**Russian:**|(1,1,1)|0.103|(1,1,1)|(3,0,1,7)|0.075
**Chinese:**|(4,0,1)|0.062|(1,1,1)|(3,0,1,7)|0.063

**SARIMAX Model** on the other hand consist of seasonal orders for Autoregression, Moving Average, Differenciation and Seasonality along with exogenous variable. With the help of SARIMAX model, MAPE score is improved for all the languages except Spanish. Which shows exogenous variable (campaigning) doesn't have impact on Spanish pageview timeseries. So final forecasting model for Spanish pageviewto be built without exogenous variable and others to be built with exogenous variable.


ARIMA and SARIMAX hyper-parameter and MAPE score comparison.

-|ARIMA|-|SARIMAX|-|-
--|:--:|:--:|:--:|:--:|:--:
**Language**|**(p,d,q)**|**MAPE**|**(p,d,q)**|**(P,D,Q,S)**|**MAPE**
**German:**|(2,1,3)|0.099|(5,1,2)|(1,0,1,7)|0.053
**English:**|(0,1,0)|0.069|(4,1,2)|(1,0,1,7)|0.063
**Spanish:**|(4,0,0)|0.138|(5,0,1)|(2,0,1,7)|0.08
**French:**|(5,0,1)|0.073|(3,1,2)|(1,0,1,7)|0.071
**Japanese:**|(2,0,1)|0.091|(5,1,2)|(3,0,1,7)|0.062
**Russian:**|(1,1,1)|0.103|(1,1,0)|(2,0,1,7)|0.074
**Chinese:**|(4,0,1)|0.062|(1,1,1)|(3,0,1,7)|0.063

**Prophet Library from facebook** slightly improves forecasting accuracy compared to SARIMAX Model but overall both models have similar performance.

MAPE for each language model with Prophet modeling is as below.

**Language**|**MAPE**
--|:--:
**German:**|0.047
**English:**|0.042
**Spanish:**|0.06
**French:**|0.05
**Japanese:**|0.062
**Russian:**|0.094
**Chinese:**|0.047